In [1]:
import numpy as np
import pandas as pd
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## Data preporation

In [2]:
import spacy
nlp=spacy.load('en_core_web_lg')
nlp.Defaults.stop_words.remove('no')
nlp.Defaults.stop_words.remove('not')
nlp.Defaults.stop_words.remove('none')
nlp.Defaults.stop_words.remove('noone')
nlp.Defaults.stop_words.remove('back')
nlp.Defaults.stop_words.add('doctors')
from preprocessing import delete_extra

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-06-12 13:21:01.838949: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-12 13:21:01.839411: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


##### Snomed preprocessing

In [3]:
concepts=pd.read_csv('CONCEPT.csv', on_bad_lines='skip', delimiter="\t", low_memory=False)

In [4]:
concepts=concepts[(concepts['vocabulary_id']=='SNOMED') & (concepts['standard_concept']=='S')]
concepts['concept_name']=concepts['concept_name'].replace({'X]': 'X] '}, regex=True).str.lower()#TODO put it to preprocessing part
concepts = concepts[['concept_id', 'concept_name']] 
concepts_id=concepts['concept_id'].tolist()
concept_names= list(map(str, concepts['concept_name']))

In [5]:
#FOR TRAINING
concepts_id=concepts_id[:20]
concept_names=concept_names[:20]

In [6]:
upd_concept_names=delete_extra(concept_names, nlp)

##### HCHS preprocessing

In [7]:
terms=pd.read_excel('concept_evaluation_new.xlsx')

In [8]:
terms=terms['sourceName'].dropna(axis=0)

In [9]:
terms=terms.str.lower().tolist()

In [10]:
upd_terms=delete_extra(terms, nlp)

### TF-IDF

In [11]:
from utils import make_flatten_list_of_terms
from tf_idf import Mapper

In [15]:
chunk_size=3#number of chunks
num=3#number of guesses
terms_concepts={'terms':upd_terms, 'concepts': upd_concept_names} 
df=pd.DataFrame()
df['Term']=make_flatten_list_of_terms(upd_terms, num)
df['Concept']= np.nan
df['Concept_id']=np.nan
df['Similarity']=np.nan

In [16]:
for i in range(0, len(upd_concept_names), chunk_size): #np.array_split(small_chunk, chunk_size)
    chunk_names=upd_concept_names[i:i+chunk_size]
    chunk_id=concepts_id[i:i+chunk_size]
    terms_concepts={'terms':upd_terms, 'concepts': chunk_names}
    document_list=terms_concepts['terms']+terms_concepts['concepts']
    model=Mapper(num, document_list, terms_concepts, chunk_id, df)
    df=model.map()
    df['Concept_id']=df['Concept_id'].astype(int)
    


In [18]:
df

,Term,Concept,Concept_id,Similarity
0,1 rest heart frequency,radiate chest pain,4001098,0.0
1,1 rest heart frequency,urine tryptophan creatinine ratio,37392117,0.0
2,1 rest heart frequency,nu hope belt leave small 6460c 70cm-78 cm leng...,46139744,0.0
3,2 rest heart frequency,radiate chest pain,4001098,0.0
4,2 rest heart frequency,urine tryptophan creatinine ratio,37392117,0.0
...,...,...,...,...
235,weight reduction,urine tryptophan creatinine ratio,37392117,0.0
236,weight reduction,nu hope belt leave small 6460c 70cm-78 cm leng...,46139744,0.0
237,yogurt,radiate chest pain,4001098,0.0
238,yogurt,urine tryptophan creatinine ratio,37392117,0.0


In [ ]:
df.to_csv('result.csv')